## Fine-tuning Gemma

## Gemma

In [ ]:
!pip install transformers datasets peft bitsandbytes accelerate openpyxl

In [3]:
!pip install transformers datasets accelerate peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 46.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [peft]1/2 [peft]


In [5]:
# ========= Step 1 =========
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig, set_seed
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split

# ========= Step 2: Configuration =========
model_name = "google/gemma-7b-it"
output_dir = "./finetuned_gemma-7b-it"
max_length = 128
batch_size = 4
gradient_accumulation = 4  # → effective batch size = 16
num_epochs = 3
seed = 42

# ========= Step 3: Load and prepare dataset =========
df = pd.read_excel("final_cleaned_8330.xlsx")[["cleaned_parent", "cleaned_reply"]].dropna()
df = df.rename(columns={"cleaned_parent": "post", "cleaned_reply": "reply"})

# Add prompt template
df["text"] = df.apply(
    lambda row: f"<|startoftext|>Assume you are a teacher or student in an online course forum. Please reply to this post: <Post> {row['post']} <Reply> {row['reply']}<|endoftext|>",
    axis=1
)

# Split into training and validation sets
train_texts, eval_texts = train_test_split(df["text"], test_size=0.2, random_state=seed)
dataset = DatasetDict({
    "train": Dataset.from_dict({"text": train_texts.tolist()}),
    "validation": Dataset.from_dict({"text": eval_texts.tolist()})
})

# ========= Step 4: Load and configure tokenizer =========
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Add pad_token

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=max_length)

tokenized_dataset = dataset.map(tokenize, batched=True)

# ========= Step 5: Prepare QLoRA model =========
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

# Optional: load without quantization
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto",
#     trust_remote_code=True
# )

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# ========= Step 6: Set training arguments and train =========
set_seed(seed)
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation,
    num_train_epochs=num_epochs,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    logging_steps=50,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

trainer.train()

# ========= Step 7: Save model and tokenizer =========
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


Map: 100%|██████████| 1660/1660 [00:00<00:00, 10681.08 examples/s]


KeyboardInterrupt: 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pandas as pd
from tqdm import tqdm

# ========= Step 1: Load the fine-tuned model =========
model_path = "./finetuned_gemma-7b-it"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# ========= Step 2: Load the original post data and run inference =========
df = pd.read_excel("final_cleaned_8330.xlsx")[["cleaned_parent", "cleaned_reply"]].dropna()
df = df.rename(columns={"cleaned_parent": "post", "cleaned_reply": "true_reply"})

generated_replies = []
for post in tqdm(df["post"], desc="Generating..."):
    prompt = f"<|startoftext|>Assume you are a teacher or student in an online course forum. Please reply to this post: <Post> {post} <Reply>"
    output = generator(prompt, max_new_tokens=80, temperature=0.7)[0]["generated_text"]
    reply = output.split("<Reply>")[-1].split("<|endoftext|>")[0].strip()
    generated_replies.append(reply)

df["generated_reply"] = generated_replies
df.to_excel("gemma_generated_replies.xlsx", index=False)


## Fine-tuning GPT-2

#  Fine-tuning a GPT-2 Replies-Generating Model w/ GPU For Free

by [Max Woolf](http://minimaxir.com)
modified by Zifeng Liu

*Last updated: October 17th, 2024*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:
0. Prepare the Mooc Posts Dataset(https://datastage.stanford.edu/StanfordMoocPosts/; Contact paepcke@cs.stanford.edu to get the data)
1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [2]:
!python3.11 -m pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1


In [3]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
# from google.colab import files

## GPU - eight NVIDIA A100 GPUs utilizing Python 3

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [4]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first.

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk (We used this one!!).
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [4]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, ?it/s]                                                         
Fetching encoder.json: 1.05Mit [00:01, 769kit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, ?it/s]                                                       
Fetching model.ckpt.data-00000-of-00001: 498Mit [02:03, 4.03Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, ?it/s]                                                   
Fetching model.ckpt.meta: 1.05Mit [00:00, 1.95Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 1.83Mit/s]                                                      


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [4]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [3]:
# file_name = "posts_data.txt"
file_name = "./data.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies.

In [4]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )



Loading checkpoint models\124M\model.ckpt
INFO:tensorflow:Restoring parameters from models\124M\model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:05<00:00,  5.89s/it]


dataset has 2568833 tokens
Training...
[10 | 172.51] loss=3.13 avg=3.13
[20 | 389.14] loss=3.35 avg=3.24
[30 | 610.21] loss=3.07 avg=3.18
[40 | 821.72] loss=3.07 avg=3.15
[50 | 1045.59] loss=3.19 avg=3.16
[60 | 1269.41] loss=3.31 avg=3.19
[70 | 1480.31] loss=3.09 avg=3.17
[80 | 1689.91] loss=3.05 avg=3.16
[90 | 1896.71] loss=3.17 avg=3.16
[100 | 2093.66] loss=3.30 avg=3.17
[110 | 2280.89] loss=3.16 avg=3.17
[120 | 2477.11] loss=3.07 avg=3.16
[130 | 2679.44] loss=3.11 avg=3.16
[140 | 2896.41] loss=3.07 avg=3.15
[150 | 3041.04] loss=3.16 avg=3.15
[160 | 3190.72] loss=3.15 avg=3.15
[170 | 3388.85] loss=2.86 avg=3.13
[180 | 3557.09] loss=2.81 avg=3.11
[190 | 3721.12] loss=3.09 avg=3.11
[200 | 3886.12] loss=3.15 avg=3.11
======== SAMPLE 1 ========
___name__
"I like the question about what is called from a different period in the history of human rights to the point that it allows people to talk about some of the human rights issues that were discussed.  What are your thoughts on it?"
"It is

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [9]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [3]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [3]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')


Loading checkpoint checkpoint\run1\model-1000
INFO:tensorflow:Restoring parameters from checkpoint\run1\model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [4]:
gpt2.generate(sess, run_name='run1')

"In the first week, one of the essays was very unhelpful. I didn't like the language, but my colleagues had the same issue. I think this may be due to the way the instructor did it. I can't blame her for that. She didn't approach the text with the exact same eyes, but she approached it with the same intent. She didn't give the reader a sense of competence, but she made a point to make sure that the author didn't make a mistake. _x0007_I think that I am going to have to get over this and read the essay again. _x0007_I was not happy with where I was on the essay, but I think I have to get over it somehow. I also have to read the essay again because I have already finished the essay. As you may know, I am not following the course closely. I am looking forward to reading more essays and I hope that I can make it through the course."
"I think the way you approach the peer grading is a problem that you have to deal with at your own pace. I think that the discussion is really important. You w

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [5]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="the pervasive idea that to be good at Math, is somehow, to be an otherwise absent-minded, Geeky Genius who can't relate to \regular people.",
              nsamples=5,
              batch_size=5,
              truncate="<|endoftext|>",
              include_prefix = False,
              return_as_list=True
              )

['the pervasive idea that to be good at Math, is somehow, to be an otherwise absent-minded, Geeky Genius who can\'t relate to \regular people.  _x0007__x0007_I also think that some of the higher-school students who have been told they are good at Math are often the ones who are no longer \\""math-dumb\\"" and are forced to think of Math as something that is \\""easy\\"".   It\'s hard to \\""get\\"" what you are actually learning."""\n"Hello everyone,_x0007__x0007_I am an elementary school teacher in a rural, predominantly African-American school district in Kansas City, MO. I teach 4-year-olds who have been born in the United States and have been taught in a different culture from their family. I am hoping to create a safe environment where all students feel safe enough to express their opinions about math, as well as some of the positive stories I hope to share._x0007__x0007_I am taking this course to make it easier for kids to feel comfortable with math. I am excited about how it wil

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [6]:
import pandas as pd

In [9]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

with open('data.txt', 'r', encoding='utf-8') as file:
    prefixes = [line.strip() for line in file.readlines() if line.strip()]

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['Prefix', 'Generated Text'])
i = 0
for prefix in prefixes:
    # Generate text using the loaded GPT-2 model
    generated_text = gpt2.generate(sess,
                                   length=500,
                                   temperature=0.7,
                                   prefix=prefix,
                                   nsamples=1,  # We want only one sample per prefix
                                   batch_size=1,  # Generate one text at a time
                                   return_as_list=True
                                   )[0]  # Since nsamples=1, take the first item
    
    if(i%100==0):
        print(generated_text,'\n')
    
    i = i + 1

    # Create a new DataFrame for the current row
    new_row = pd.DataFrame({'Prefix': [prefix], 'Generated Text': [generated_text]})

    # Concatenate the new row to the existing DataFrame
    df = pd.concat([df, new_row], ignore_index=True)

# Set the filename with the current timestamp
filename = f'gpt2_gentext0.7_{datetime.utcnow():%Y%m%d_%H%M%S}.xlsx'
df.to_excel(filename, index=False)

# gpt2.generate_to_file(sess,
#                       destination_path=gen_file,
#                       length=500,
#                       temperature=0.7,
#                       nsamples=100,
#                       batch_size=20
#                       )

Interesting! How often we say those things to others without really understanding what we are saying. That must have been a powerful experience! Excellent! _x0007__x0007_It's interesting to note that all culture is about the nurturing of children. I don't know how we can get all the more children to be nurtured if we are talking about helping other children. It's not just a one-way street, but it is important to be able to do this throughout our life. _x0007__x0007_What are some of the things you are describing and why?_x0007__x0007_1. I believe that if you are good at math, and you are using your brain, you will be better at math. You will have more opportunities to learn, and you will be more likely to start off the year better than you did. _x0007__x0007_2. The more you are exposed to math, the more you will be able to understand it more easily. I don't think you have to be a math person to understand it. There is so much that can be learned through math that you can learn from. You

MemoryError: bad allocation

In [11]:
# Set the filename with the current timestamp
filename = f'gpt2_gentext0.7_{datetime.utcnow():%Y%m%d_%H%M%S}.xlsx'
df.to_excel(filename, index=False)

In [ ]:
!pip install textstat

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 354Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 131Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 279Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:23, 131Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 380Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 226Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 199Mit/s]                                                       


In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

W0828 18:37:58.571830 139905369159552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Loading pretrained model models/774M/model.ckpt


In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

The secret of life is that it's really easy to make it complicated," said Bill Nye, the host of the popular science show "Bill Nye the Science Guy." "And this is one of the reasons why we all need to be smarter about science, because we can't keep up with the amazing things that are going on all the time."

While Nye is correct that "everything that's going on all the time" is making the world a better place, he misses the point. This is not
The secret of life is in the rhythm of the universe. It's not a mystery. It's not a mystery to me. It's the nature of the universe. It's the beauty of the universe. It's the way the universe works. It's the way the universe is. It's the way the universe is going to work. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way
The secret of life is in the universe.


-

The Red Devil

It's the end of the world as we know it, and the only thing that can save us is a band of 

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Fine-tuning GPT-2 small for generating replies for online discussion

## Task list 

### Read cleaned_parent (post) and cleaned_reply (reply) columns from final_cleaned_8330.xlsx file

### Split into fine-tune training set and validation set by 80% / 20

### Train GPT-2 small using Hugging Face

### After training, the fine-tuned model is used to batch inference all 8330 posts to generate replies

In [5]:
!pip install transformers datasets accelerate openpyxl

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.9/30.9 MB 34.7 MB/s eta 0:00:00a 0:00:01
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.1
    Uninstalling tqdm-4.66.1:
      Successfully uninstalled tqdm-4.66.1
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [datasets]6/7 [datasets]


In [1]:
from transformers import set_seed
set_seed(42)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 1 Fine-tuning

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from sklearn.model_selection import train_test_split

# Read and clean the data
df = pd.read_excel("final_cleaned_8330.xlsx")
df = df[["cleaned_parent", "cleaned_reply"]].dropna()
df = df.rename(columns={"cleaned_parent": "post", "cleaned_reply": "reply"})

# Construct prompt-style training text (core formatting)
df["text"] = df.apply(
    lambda row: f"<|startoftext|>Assume you are a teacher or student in an online course forum. Please reply to this post: <Post> {row['post']} <Reply> {row['reply']}<|endoftext|>",
    axis=1
)

# Split into training and validation sets (80% / 20%)
train_texts, eval_texts = train_test_split(df["text"], test_size=0.2, random_state=42)
train_dataset = Dataset.from_dict({"text": train_texts.tolist()})
eval_dataset = Dataset.from_dict({"text": eval_texts.tolist()})
dataset = DatasetDict({"train": train_dataset, "validation": eval_dataset})

# Load GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({
    "bos_token": "<|startoftext|>",
    "eos_token": "<|endoftext|>",
    "pad_token": "<|pad|>"
})

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

# Tokenize the dataset
def tokenize_fn(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_fn, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2_post_reply_prompt_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_total_limit=2,
    logging_steps=50,
    logging_dir="./logs",
    fp16=False  # Set to True if using GPU with fp16 support
)

# Create Trainer
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Fine-tune the model
trainer.train()

# Save the model and tokenizer
model_path = "./gpt2_post_reply_prompt_finetuned"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)


Map: 100%|██████████| 1660/1660 [00:01<00:00, 1378.95 examples/s]
/opt/homebrew/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/kr/1ch_8lcx0zz7sztc44474v0r0000gn/T/ipykernel_90834/1149708173.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
from tqdm import tqdm

# Load the fine-tuned model
model_path = "./gpt2_post_reply_prompt_finetuned"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Read all post data
df = pd.read_excel("final_cleaned_8330.xlsx")
df = df[["cleaned_parent", "cleaned_reply"]].dropna()
df = df.rename(columns={"cleaned_parent": "post", "cleaned_reply": "true_reply"})

# Construct prompt and generate replies
generated_replies = []

for post in tqdm(df["post"], desc="Generating replies"):
    prompt = f"<|startoftext|>Assume you are a teacher or student in an online course forum. Please reply to this post: <Post> {post} <Reply>"
    output = generator(prompt, max_new_tokens=80, temperature=0.7)[0]["generated_text"]
    
    # Extract reply segment (starting after <Reply>)
    reply = output.split("<Reply>")[-1].split("<|endoftext|>")[0].strip()
    generated_replies.append(reply)

# Add result column and save
df["generated_reply"] = generated_replies
df.to_excel("gpt2_generated_replies.xlsx", index=False)


## Fine-tuning LLaMA

## Llama fintune

In [ ]:
# ========= Step 1=========
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig, set_seed
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split

# ========= Step 2=========
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
output_dir = "./finetuned_llama3-8b"
max_length = 128
batch_size = 4
gradient_accumulation = 4  # → effective batch size = 16
num_epochs = 3
seed = 42

# ========= Step 3 =========
df = pd.read_excel("final_cleaned_8330.xlsx")[["cleaned_parent", "cleaned_reply"]].dropna()
df = df.rename(columns={"cleaned_parent": "post", "cleaned_reply": "reply"})

df["text"] = df.apply(
    lambda row: f"<|startoftext|>Assume you are a teacher or student in an online course forum. Please reply to this post: <Post> {row['post']} <Reply> {row['reply']}<|endoftext|>",
    axis=1
)

train_texts, eval_texts = train_test_split(df["text"], test_size=0.2, random_state=seed)
dataset = DatasetDict({
    "train": Dataset.from_dict({"text": train_texts.tolist()}),
    "validation": Dataset.from_dict({"text": eval_texts.tolist()})
})

# ========= Step 4: Tokenizer =========
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=max_length)

tokenized_dataset = dataset.map(tokenize, batched=True)

# ========= Step 5: LoRA =========
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# ========= Step 6: =========
set_seed(seed)

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation,
    num_train_epochs=num_epochs,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    logging_steps=50,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

trainer.train()

# ========= Step 7=========
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pandas as pd
from tqdm import tqdm

# ========= Step 1=========
model_path = "./finetuned_llama3-8b"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# ========= Step 2 =========
df = pd.read_excel("final_cleaned_8330.xlsx")[["cleaned_parent", "cleaned_reply"]].dropna()
df = df.rename(columns={"cleaned_parent": "post", "cleaned_reply": "true_reply"})

generated_replies = []
for post in tqdm(df["post"], desc="Generating..."):
    prompt = f"<|startoftext|>Assume you are a teacher or student in an online course forum. Please reply to this post: <Post> {post} <Reply>"
    output = generator(prompt, max_new_tokens=80, temperature=0.7)[0]["generated_text"]
    reply = output.split("<Reply>")[-1].split("<|endoftext|>")[0].strip()
    generated_replies.append(reply)

df["generated_reply"] = generated_replies
df.to_excel("llama3_generated_replies.xlsx", index=False)
